# Exercício de Análise de Dados 
*** 
<br>

### Cálculo da soma de remunerações em um dataframe a partir de uma lista importada de arquivos PDF.
<br>

<br>

### Importação dos Módulos

<br>

In [2]:
#utilizei o camelot pela sintaxe simples e eficiência
import os
import numpy as np
import pandas as pd
import camelot
import zipfile

<br>

### Importação dos Arquivos PDF e Concatenação da Listagem
<br>
Obs. O Camelot possui um bug: a seleção de várias páginas, (ex. pages='1-3' ou 'all'), não funciona. Então foi necessaŕio repetir a rotina de extração para cada página.

<br>

Isso também fez com que a rotina não funcionasse para o último arquivo, que só possui uma página, gerando erro de índice e obrigando a tratar esse último arquivo separadamente. 
<br>
<br>

In [9]:
List_Path = './Lista_Completa/Arquivos/'
Ult_Path = './Lista_Completa/ult/'
Lista_Completa = pd.DataFrame()
df3 = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8])

# Leitura dos arquivos de 3 páginas
for arq in os.listdir(List_Path):
    df1 = camelot.read_pdf(List_Path+arq, pages='1') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    df1 = camelot.read_pdf(List_Path+arq, pages='2') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    df1 = camelot.read_pdf(List_Path+arq, pages='3') 
    df2 = df1[0].df
    df3 = df3.append(df2)
    
#Leitura do último arquivo com uma só página
for arq in os.listdir(Ult_Path):
    df1 = camelot.read_pdf(Ult_Path+arq, pages='1')
    df2 = df1[0].df
    df3 = df3.append(df2)

df3.rename(columns=df3.iloc[0]).drop(df3.index[0])
df3.columns = ['A', 'B', 'C', 'NOME', 'E', 'F', 'G', 'H', 'I']
df3.drop(df3[['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']], axis=1, inplace=True)
df3.drop(df3.index[0], axis=0, inplace=True)

#Criação da Lista Completa
Lista_Completa = Lista_Completa.append(df3)
Lista_Completa

,NOME
1,TATIANA CASTELO BRANCO SILVA LOPES
2,TATIANA FERRAZ LINS
3,TATIANA MARINS FARIAS
4,TATIANA OLIVEIRA SILVA
5,TATIANA OLIVEIRA SOUZA
...,...
5,YURI SILVA SERAFIM MACHADO
6,ZENAR MARIA RIBEIRO MENDES DE\nSABOIA
7,ZILMA VIEIRA DA CRUZ
8,ZILOCI FERREIRA DOS SANTOS


<br>

### Explorando o Arquivo Geral zipado

<br>

In [3]:
with zipfile.ZipFile('Servidores.zip') as BigFile:
    print(BigFile.namelist(),sep='\n')

['Servidores/', 'Servidores/202001_Remuneracao.csv']


<br>

### Preparação do Arquivo selecionado dentro do zip como DataFrame

<br>

In [26]:
with zipfile.ZipFile('Servidores.zip') as BigFile:
    with BigFile.open('Servidores/202001_Remuneracao.csv') as data:
        dfBig = pd.read_csv(data, sep=';', decimal=",", low_memory=False, encoding='ISO-8859-1') #testar com nrows=10000 primeiro

<br>

### Filtragem dos Dados

<br>

In [11]:
dfBig.columns

Index(['ANO', 'MES', 'Id_SERVIDOR_PORTAL', 'CPF', 'NOME',
       'REMUNERAÇÃO BÁSICA BRUTA (R$)', 'REMUNERAÇÃO BÁSICA BRUTA (U$)',
       'ABATE-TETO (R$)', 'ABATE-TETO (U$)', 'GRATIFICAÇÃO NATALINA (R$)',
       'GRATIFICAÇÃO NATALINA (U$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (R$)',
       'ABATE-TETO DA GRATIFICAÇÃO NATALINA (U$)', 'FÉRIAS (R$)',
       'FÉRIAS (U$)', 'OUTRAS REMUNERAÇÕES EVENTUAIS (R$)',
       'OUTRAS REMUNERAÇÕES EVENTUAIS (U$)', 'IRRF (R$)', 'IRRF (U$)',
       'PSS/RPGS (R$)', 'PSS/RPGS (U$)', 'DEMAIS DEDUÇÕES (R$)',
       'DEMAIS DEDUÇÕES (U$)', 'PENSÃO MILITAR (R$)', 'PENSÃO MILITAR (U$)',
       'FUNDO DE SAÚDE (R$)', 'FUNDO DE SAÚDE (U$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (R$)',
       'TAXA DE OCUPAÇÃO IMÓVEL FUNCIONAL (U$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$)',
       'REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$)',
       'VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*)',
       'VERBAS INDENIZATÓ

In [16]:
#Extração do subconjunto de dados
dfSEL = dfBig.loc[dfBig['NOME'].isin(Lista_Completa['NOME'])] 
dfSEL

,ANO,MES,Id_SERVIDOR_PORTAL,CPF,NOME,REMUNERAÇÃO BÁSICA BRUTA (R$),REMUNERAÇÃO BÁSICA BRUTA (U$),ABATE-TETO (R$),ABATE-TETO (U$),GRATIFICAÇÃO NATALINA (R$),...,REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (R$),REMUNERAÇÃO APÓS DEDUÇÕES OBRIGATÓRIAS (U$),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - CIVIL (U$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (R$)(*),VERBAS INDENIZATÓRIAS REGISTRADAS EM SISTEMAS DE PESSOAL - MILITAR (U$)(*),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (R$),VERBAS INDENIZATÓRIAS PROGRAMA DESLIGAMENTO VOLUNTÁRIO  MP 792/2017 (U$),TOTAL DE VERBAS INDENIZATÓRIAS (R$)(*),TOTAL DE VERBAS INDENIZATÓRIAS (U$)(*)
75,2020,1.0,2715680.0,***.856.736-**,ABDO HASSEN SALMAN,19280.76,0.0,0.0,0.0,0.0,...,13549.19,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
830,2020,1.0,5205493.0,***.754.214-**,ADAILTON ABREU DA SILVA,17579.95,0.0,0.0,0.0,0.0,...,12336.16,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
1320,2020,1.0,101329.0,***.147.267-**,ADALTON FARIA,20468.03,0.0,0.0,0.0,0.0,...,14167.92,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
2484,2020,1.0,3805496.0,***.458.016-**,ADEMILDES FERREIRA DE LIMA,17952.92,0.0,0.0,0.0,0.0,...,12526.81,0.0,680.70,0.0,0.0,0.0,0.0,0.0,680.70,0.0
2591,2020,1.0,4706421.0,***.136.944-**,ADEMIR CARNEIRO DA CUNHA,21099.50,0.0,0.0,0.0,0.0,...,14535.96,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553023,2020,1.0,3508477.0,***.088.001-**,YURI JOSE SILVA NASCIMENTO,18985.54,0.0,0.0,0.0,0.0,...,13200.60,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
553104,2020,1.0,7306222.0,***.508.698-**,YURI ROBERTO PEREIRA,17372.34,0.0,0.0,0.0,0.0,...,13220.96,0.0,458.00,0.0,0.0,0.0,0.0,0.0,458.00,0.0
553120,2020,1.0,1902744.0,***.381.705-**,YURI SILVA SERAFIM MACHADO,20468.03,0.0,0.0,0.0,0.0,...,16477.96,0.0,779.00,0.0,0.0,0.0,0.0,0.0,779.00,0.0
554034,2020,1.0,415960.0,***.251.256-**,ZILMA VIEIRA DA CRUZ,21099.50,0.0,0.0,0.0,0.0,...,14581.98,0.0,577.28,0.0,0.0,0.0,0.0,0.0,577.28,0.0


In [27]:
#Filtragem inicial do campo a ser processado
dfWork = pd.DataFrame(dfSEL['REMUNERAÇÃO BÁSICA BRUTA (R$)'])
dfWork

,REMUNERAÇÃO BÁSICA BRUTA (R$)
75,19280.759766
830,17579.949219
1320,20468.029297
2484,17952.919922
2591,21099.500000
...,...
553023,18985.539062
553104,17372.339844
553120,20468.029297
554034,21099.500000


<br>

### Eliminando registros inválidos

<br>

In [21]:
dfWork.loc[dfWork["REMUNERAÇÃO BÁSICA BRUTA (R$)"].isnull()]  #verificando valores nulos para elaborar a melhor estratégia

,REMUNERAÇÃO BÁSICA BRUTA (R$)


In [8]:
#Não há valores nulos

In [24]:
Transformando em dfloat32 para economizar memória no processamento
dfWork['REMUNERAÇÃO BÁSICA BRUTA (R$)'].astype(np.float32)

75        19280.759766
830       17579.949219
1320      20468.029297
2484      17952.919922
2591      21099.500000
              ...     
553023    18985.539062
553104    17372.339844
553120    20468.029297
554034    21099.500000
554070    15536.870117
Name: REMUNERAÇÃO BÁSICA BRUTA (R$), Length: 3295, dtype: float32

<br>

### Soma Final da Remuneração dos Servidores da Lista

<br>

In [28]:
SOMA = dfWork['REMUNERAÇÃO BÁSICA BRUTA (R$)'].sum()
print("A soma de todas as reminerações da lista é: ")
print(SOMA)

A soma de todas as reminerações da lista é: 
63661310.0
